In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay



In [152]:

pd.set_option('display.max_columns', None)

In [153]:
trainset = pd.read_csv('./Dataset/LoanPrediction_TrainSet.csv')
testset = pd.read_csv('./DataSet/LoanPrediction_TestSet.csv')

trainset = trainset.drop(columns=["Unnamed: 0"])
testset = testset.drop(columns=["Unnamed: 0"])







In [154]:
trainset.head(10)

,LoanID,Gender,Married,Dependents,Education,SelfEmployed,ApplicantIncome,CoapplicantIncome,LoanAmount,LoanAmountTerm,CreditHistory,PropertyArea,LoanStatus
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [166]:

ABT_train = trainset.describe(include="all").T

ABT_train = ABT_train.reset_index().rename(columns={'index': 'Feature'})
ABT_train['Missing'] = trainset.isnull().sum().values
ABT_train['dtype'] = trainset.dtypes.values
ABT_train


# Wrong data types for some features like 'Dependents' and 'Credit_History' and missing values in some features
# There is some unbalance in the target variable 'Loan_Status' as i belive aswell som Bias in features for example Gender more male and females
# Dependents looks like it have some not allowed additions sign as well for 3+ as a category which i will fix


,Feature,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Missing,dtype
0,LoanID,614,614,LP001002,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,object
1,Gender,601,2,Male,489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,object
2,Married,611,2,Yes,398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,object
3,Dependents,599,4,0,345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,object
4,Education,614,2,Graduate,480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,object
5,SelfEmployed,582,2,No,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,object
6,ApplicantIncome,614.0,NaN,NaN,NaN,5403.459283,6109.041673,150.0,2877.5,3812.5,5795.0,81000.0,0,int64
7,CoapplicantIncome,614.0,NaN,NaN,NaN,1621.245798,2926.248369,0.0,0.0,1188.5,2297.25,41667.0,0,float64
8,LoanAmount,592.0,NaN,NaN,NaN,146.412162,85.587325,9.0,100.0,128.0,168.0,700.0,22,float64
9,LoanAmountTerm,600.0,NaN,NaN,NaN,342.0,65.12041,12.0,360.0,360.0,360.0,480.0,14,float64


In [167]:
ABT_train.to_excel('ABT_train.xlsx', index=False)

TypeError: ExcelFormatter.write() got an unexpected keyword argument 'engine_kwargs'

In [165]:
pd.DataFrame().to_excel(trainset, sheet_name='Sheet1')

TypeError: ExcelFormatter.write() got an unexpected keyword argument 'engine_kwargs'